In [1]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
import xarray as xa

from typing import List

from IPython.core.display import display, HTML

np.set_printoptions(linewidth=200)
pd.options.display.max_columns = None
display(HTML("<style>.container { width:100% !important; }</style>"))

# Part A: Prepare the dataset

## A1. Load the necessary datasetes and make initial improvements

In [2]:
# load the original Kaggle Survey dataset.
orig = pd.read_csv('../data/kaggle_survey_2020_responses.csv', low_memory=False)
# the first row contains the question. This is useful to read but,
questions = orig.loc[0].reset_index(drop=True)
# It can be better read like:
# questions[21]
# The question is not the data we want to explore,
#so we discard it from the df we will work on.
df = orig.loc[1:].reset_index(drop=True)

In [3]:
# Load countries' income groups from World Bank
country_groups = pd.read_csv('../data/wb_country_income_groups.csv')
country_groups.rename(columns={'Economy': 'country', 'Income group': 'group'}, inplace=True)
country_groups.head()
country_groups.group.unique()

array(['Low income', 'Upper middle income', 'Lower middle income', 'High income'], dtype=object)

In [4]:
# Load mean nominal wage data from International Labor Organization
ilo_mean_wage = pd.read_csv('../data/ilo_mean_nom_wage_usd.csv')
ilo_mean_wage.drop(['Source', 'Sex', 'Economic activity', 'Currency'], axis=1, inplace=True)
ilo_mean_wage.rename(columns={'Country': 'country', 'Value': 'monthly_earnings', 'Year': 'year'}, inplace=True)

In [5]:
# set an approximation for India
# https://www.numbeo.com/cost-of-living/country_price_rankings?itemId=105
#in_india_wage = pd.DataFrame([['India', int(2019), int(446)]], columns=list(ilo_mean_wage.columns))
#ilo_mean_wage = ilo_mean_wage.append(in_india_wage)

ilo_mean_wage['yearly_earnings'] = ilo_mean_wage['monthly_earnings'] * 12

# sort by max year and drop duplicates
# df[df.groupby('ID')['date'].transform('max') == df['date']]
# https://stackoverflow.com/questions/53199076/grab-rows-with-max-date-from-pandas-dataframe
ilo_mean_wage = ilo_mean_wage.loc[ilo_mean_wage.groupby('country').year.idxmax()].reset_index()
ilo_mean_wage = ilo_mean_wage.set_index('country')

In [6]:
ilo_mean_wage.sort_values(by='monthly_earnings')[:6]

,index,year,monthly_earnings,yearly_earnings
country,,,,
Benin,49,2011,0.2,2.4
Kyrgyzstan,386,2018,17.9,214.8
Burundi,43,2014,41.4,496.8
Rwanda,626,2017,68.2,818.4
Madagascar,465,2015,71.4,856.8
Ethiopia,229,2013,72.4,868.8


In [7]:
# load euro usd exchange rate
usd_euro_rate = pd.read_csv('../data/ecb_usd_euro_avg_exch_rate_filtered.csv', header=None)
# usd_euro_rate.head(6)
# float(usd_euro_rate.iloc[5][1])

In [8]:
# Load mean nominal wage data from International Labor Organization
eurostat_mean_earnings = pd.read_csv('../data/eurostat_gross_earnings_euros_2019_filtered.csv')
# eurostat_mean_earnings['gross_earn_2019_euros'] = eurostat_mean_earnings['gross_2019_euros'].str.replace(',', '')
eurostat_mean_earnings['yearly_earnings'] = (eurostat_mean_earnings['gross_2019_euros'].astype(float)) * float(usd_euro_rate.iloc[5][1])
eurostat_mean_earnings = eurostat_mean_earnings.set_index('country')

In [9]:
oecd_mean_wage = pd.read_csv('../data/oecd_ann_avg_wage_2019.csv')
oecd_mean_wage = oecd_mean_wage.set_index('country')

In [10]:
oecd_mean_wage.head(3)

,yearly_earnings
country,
Australia,54401.425296
Austria,53902.949045
Belgium,55590.130461


In [11]:
# Load mean nominal wage data from International Labor Organization
numbeo_mean_wage = pd.read_csv("../data/numbeo.csv")
numbeo_mean_wage = numbeo_mean_wage.assign(year=2019)
numbeo_mean_wage['yearly_earnings'] = numbeo_mean_wage['monthly_earnings'] * 12
numbeo_mean_wage = numbeo_mean_wage.set_index('country')

In [12]:
# Is this syntantic sugar optimal?
wages_data = pd.concat([
    eurostat_mean_earnings['yearly_earnings'],
     oecd_mean_wage['yearly_earnings'],
     ilo_mean_wage['yearly_earnings'],
     numbeo_mean_wage['yearly_earnings']
],
    axis=1,
    keys=['eurostat_mean_earnings',
          'oecd_mean_wage',
          'ilo_mean_wage',
          'numbeo_mean_wage']
)


wages_data = wages_data.reset_index(level=0).rename(columns={'index': 'country'})

In [13]:
wages_data

# wages_data.country.unique()

# set(df.Q3.unique()).difference(wages_data.country.unique())

,country,eurostat_mean_earnings,oecd_mean_wage,ilo_mean_wage,numbeo_mean_wage
0,Belgium,55488.084670,55590.130461,41056.8,28795.32
1,Bulgaria,8699.880790,NaN,8299.2,7929.84
2,Czechia,17826.526175,NaN,18601.2,NaN
3,Denmark,64161.287775,57149.594306,NaN,43336.92
4,Germany,58420.771650,53637.801603,59108.4,33441.96
...,...,...,...,...,...
161,Kenya,NaN,NaN,NaN,4654.56
162,Iran,NaN,NaN,NaN,4117.20
163,Moldova,NaN,NaN,NaN,3877.44
164,Tunisia,NaN,NaN,NaN,3610.92


In [14]:
# compare_monthly_salaries = pd.merge(ilo_mean_wage.groupby("country").tail(1), numbeo_avg_earn, on=["country"])

# compare_monthly_salaries = compare_monthly_salaries.assign(
#     monthly_diff=np.abs(compare_monthly_salaries.monthly_earnings_x - compare_monthly_salaries.monthly_earnings_y)
# )

# compare_monthly_salaries = compare_monthly_salaries.assign(
#     norm_diff=compare_monthly_salaries.monthly_diff / compare_monthly_salaries[["monthly_earnings_x", "monthly_earnings_y"]
#                                                                               ].min(axis=1) * 100)
# compare_monthly_salaries.tail(3)

# compare_monthly_salaries[compare_monthly_salaries.country == 'Poland']

# compare_monthly_salaries[compare_monthly_salaries.country == 'United States']

## A2. Define useful functions 

In [15]:
# In order to make the analysis easier, we can filter the dataframe into smaller ones 
# which will contain the data of each multiple choice question.
# To this end, defining a function can be helpful
def filter_df(df, question_index):
    columns = [col for col in df.columns if col.startswith(f"Q{question_index}_")]
    df = df.loc[0:][columns].fillna(False)
    unique_values = []
    for column in columns:
        column_unique = list(df[column].unique())
        column_unique.remove(False)
        unique_values.append(column_unique[0])
    filtered_df = df.replace(to_replace=unique_values, value=True)
    filtered_df.columns = [value.strip() for value in unique_values]
    return filtered_df

# We will usually want to join one or more of the filtered dataframes to the background one:
def join_dfs(*dataframes: List[pd.DataFrame]) -> pd.DataFrame:
    df = pd.concat(dataframes, axis=1, ignore_index=False)
    return df

In [16]:
## Normalize data format
# Strip whitespace from all the anwers
# Note: Since, initially, all the columns are of dtype "object" We could do this simply with: 
#           df = df.apply(lambda x: x.str.strip())
#       Nevertheless, we want to convert one column to a numeric one, so reexecuting this cell will be failing.
#       Thus we will do this in a bit more elaborate way.
string_cols = df.select_dtypes(object).columns
df[string_cols] = df[string_cols].apply(lambda x: x.str.strip())

## A3 Modify Kaggle data format to assist data exploration (column names, answers)

In [17]:
# Column 1: "Time from Start to Finish (seconds)" contains integers. Let's cast it and rename it to something more convenient
df.rename(columns={'Time from Start to Finish (seconds)': 'duration'}, inplace=True)
df['duration'] = df['duration'].apply(int)

In [18]:
# orig.Q24.unique() # Q24 salary
# Remove symbols and "," from salary ranges.
df.Q24 = df.Q24.replace({
    '$0-999': '0-999',
    '> $500,000': '500,000-999,999',
    '300,000-500,000': '300,000-499,999', 
}).str.replace(',', '')

#### Columns about experience have different ranges and different format. Modify format to be similar and DNRY
#### This way, we minimize errors that may be caused by human typing,
#### e.g. executive summary p. 10, machine learning experience class from 10-20 years is reference as 10-15 years

In [19]:
orig.Q6.unique(), orig.Q15.unique()

(array(['For how many years have you been writing code and/or programming?', '5-10 years', '10-20 years', '3-5 years', '< 1 years', '1-2 years', '20+ years', 'I have never written code', nan],
       dtype=object),
 array(['For how many years have you used machine learning methods?', '1-2 years', 'I do not use machine learning methods', '3-4 years', nan, 'Under 1 year', '2-3 years', '4-5 years', '5-10 years',
        '20 or more years', '10-20 years'], dtype=object))

In [20]:
df.Q6 = df.Q6.replace({
    '< 1 year': '< 1',
    'I have never written code': '0'
}).str.replace(' years', '')

In [21]:
df.Q15 = df.Q15.replace({
    'Under 1 year': '< 1',
    '20 or more years': '20+',
    'I do not use machine learning methods': '0'
}).str.replace(' years', '')

In [22]:
# Refine Company employment size values
df.Q20 = df.Q20.replace({
    '10,000 or more employees': '> 10000',
}).str.replace(' employees', '').replace(',', '')

In [23]:
df.Q3 = df.Q3.replace({
    'Russia': 'Russian Federation',
    'United States of America': 'United States',
    'United Kingdom of Great Britain and Northern Ireland': 'United Kingdom',
    'Iran, Islamic Republic of...': 'Iran',
    'Republic of Korea': 'South Korea'
})

In [24]:
# Select useful columns for data validity exploration and rename them.
VALIDATION_COLS_NAMES = {
    'Q1': 'age',
    'Q2': 'gender',
    'Q3': 'country',
    'Q4': 'education',
    'Q5': 'role',
    'Q6': 'code_exp',
    'Q15': 'ml_exp',
    'Q20': 'employees',
    'Q21': 'team_ds',
    'Q22': 'company_ml_use',
    'Q24': 'salary',
    'Q25': 'spend_ds'
}

df.rename(columns=VALIDATION_COLS_NAMES, inplace=True)

validation_cols = [*VALIDATION_COLS_NAMES.values()]

# Concatenate dataframe with wage data

In [25]:
salary_thres = pd.DataFrame(
    columns=['country', 'mean_salary', 'minimum_salary', 'below_avg_salary', 'high_to_top', 'top']
)

In [26]:
salary_thres.country = df.country.unique()
salary_thres = salary_thres.merge(wages_data, on='country', how='left')

In [27]:
# SOS remember to use same names for eurostat, oecd, ilo, numbeo
len(salary_thres)

54

In [28]:
salary_thres.head()

,country,mean_salary,minimum_salary,below_avg_salary,high_to_top,top,eurostat_mean_earnings,oecd_mean_wage,ilo_mean_wage,numbeo_mean_wage
0,Colombia,NaN,NaN,NaN,NaN,NaN,NaN,NaN,4777.2,3780.96
1,United States,NaN,NaN,NaN,NaN,NaN,57055.194355,65835.577645,50797.2,42706.20
2,Argentina,NaN,NaN,NaN,NaN,NaN,NaN,NaN,6370.8,5320.92
3,Japan,NaN,NaN,NaN,NaN,NaN,47973.754380,38617.465494,2822.4,32524.32
4,India,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5375.88


In [29]:
# https://stackoverflow.com/questions/38965667/pandas-take-whichever-column-is-not-nan
# df["COL3"] = df["COL1"].fillna(df["COL2"])
# or more speacial: https://stackoverflow.com/a/53068486

# or the other way around:
# https://stackoverflow.com/questions/37962759/how-set-values-in-pandas-dataframe-based-on-nan-values-of-another-column

In [30]:
salary_thres['mean_salary'] = salary_thres['eurostat_mean_earnings'].fillna(salary_thres['oecd_mean_wage']).fillna(salary_thres['ilo_mean_wage']).fillna(salary_thres['numbeo_mean_wage'])

# ^ ΠΟΣΟ ΜΑ ΠΟΣΟ ΓΑΜΑΕΙ ΤΟ PANDAS ! ^

In [31]:
# TO DO: create columns "source of mean salary" by stripping prefix where not null ?
salary_thres.head()

,country,mean_salary,minimum_salary,below_avg_salary,high_to_top,top,eurostat_mean_earnings,oecd_mean_wage,ilo_mean_wage,numbeo_mean_wage
0,Colombia,4777.200000,NaN,NaN,NaN,NaN,NaN,NaN,4777.2,3780.96
1,United States,57055.194355,NaN,NaN,NaN,NaN,57055.194355,65835.577645,50797.2,42706.20
2,Argentina,6370.800000,NaN,NaN,NaN,NaN,NaN,NaN,6370.8,5320.92
3,Japan,47973.754380,NaN,NaN,NaN,NaN,47973.754380,38617.465494,2822.4,32524.32
4,India,5375.880000,NaN,NaN,NaN,NaN,NaN,NaN,NaN,5375.88


In [ ]:
# # create salary thresholds by country

# salary_thres.yearly_earnings = salary_thres.country.map(mean_wage.set_index('country')['yearly_earnings'])
# # df1['value'] = df1['condition'].map(df2.set_index('condition')['value'])

# Part B: Examine the data for invalid values and clean it

## B1. Investigate duration -> Examine everything!

In [ ]:
# Some people were too quick in completing the survey and their answers should be ignored.
# Nevertheless, coming up with a reasonable "cut-off" threshold is not that easy though.
df.duration.nsmallest(200, keep='all')

## B2. Participants who did not answer any non-demographic questions.
### Let's identify them.

In [ ]:
questions[:7]

In [ ]:
# Note: The non-demographic questions start from Q7 and afterwards (years of coding (or not) is included in basic demographic questions here),
# that's why we use:
#     df.iloc[:, 6:]
temp_df = df.iloc[0:, 7:]
only_answer_demographic = ((temp_df == 'None')
                           | temp_df.isnull()).all(axis=1)

len(df[only_answer_demographic])

# Construct logical conditions to filter out invalid data:

In [ ]:
# assume experience is universal across countries
# although in fact experience is more scarce in developing countries
code_exp_levels = {
    'minimum': ['0', '< 1'],
    'basic': ['2-3', '3-5'],
    'intermediate': ['5-10'],
    'advanced': ['10-20', '20+']
}


ml_exp_levels = {
    'minimum': ['0', '< 1'],
    'basic': ['2-3'],
    'intermediate': ['3-4', '4-5'],
    'advanced': ['5-10', '10-20', '20+']
}



# Condition 1 participants with All Nan or Nonw after Q6:
only_answer_demographic = (
    (temp_df == 'None')
    | temp_df.isnull()
).all(axis=1)

only_answer_demographic.sum()



# Condition 2 too young for top experience or top salary
extreme_young = (
    (df.age <= '24')
    & ((df.code_exp == '20+')
       | (df.ml_exp == '20+')
       | (df.salary.isin(['300000-499999', '500000-999999']))
      )
)
# extreme_young.sum()
# len(df[extreme_young])


#######
## Will add World Bank income groups for countries in conditions below 
## ['Low income', 'Upper middle income', 'Lower middle income', 'High income']
## Then: group salaries ['minimum', 'subsistence or below mean', 'mean', 'middle to top', 'top']
## => we will have a very loose combination of saralies for 4 countries income groups
### Perhaps map countries with other datasets.

# Condition 3 too much experience for minimum salary
too_exper_for_min_salary = (
    (df.salary.isin(['0-999'])
    & (df.code_exp.isin(['3-5', '5-10', '10-20', '20+'])
       | (df.ml_exp.isin(['2-3', '3-4', '4-5', '5-10', '10-20', '20+']))
      ))
)

# len(df[too_exper_for_min_salary])

# Condition 4 too much experience for below average salary
too_exper_for_subsistence_salary = (
    (df.salary.isin(['1000-1999', '2000-2999', '3000-3999', '4000-4999']))
                             #  & (df.role != 'Currently not employed')  # not necessary condition
                               & ((df.code_exp.isin(['5-10', '10-20', '20+'])
                                  | (df.ml_exp.isin(['5-10', '10-20', '20+'])))
                            ))

#len(df[too_exper_for_subsistence_salary])

# Condition 5 too much experience for average salary
too_exper_for_avg_salary = (
    #(df.salary.isin(['5000-7499', '7500-9999']))
        (df.salary.isin(['5000-7499']))
                               #& (df.role != 'Currently not employed')  # not necessary condition
                               & ((df.code_exp.isin(['10-20', '20+'])
                                  | (df.ml_exp.isin(['10-20', '20+'])))
                            ))

# len(df[too_exper_for_avg_salary])

# Group all possible invalidator conditions in cell above !

In [ ]:
# # So there is a large number of 1082, more than 5% of participants who did not answer anything related to data science.
# # Let's drop these participants. 
# if len(only_answer_demographic) != len(df):
#     print("rows already dropped!")
# else:
#     df = df[~only_answer_demographic].reset_index(drop=True)

In [ ]:
df = df[~only_answer_demographic].reset_index(drop=True)

In [ ]:
len(df)

In [ ]:
# After the drops, the duration seems to be much more reasonable. 
# Using a threshold might still make some sense, but probably we may drop some real typing heros.
# Any suggestions on how to further investigate this?
df.duration.nsmallest(100, keep='all')
# df[df.duration < 120]

In [ ]:
len(df[df.duration < 61])

In [ ]:
# We could also set a more strict quality limit
# e.g. keep participants for minimum 4 answers besides demographics = drop 600 more.
# Anyway, we shall keep those and find other methods to filter out invalid data.
len(df.dropna(thresh=11))

## B3 Discard invalid submissions, using logical conditions.
### Not outliers, we just drop data entries who are invalid.

### First easy candidate for invalid submissions: too young for experience or salary

In [ ]:
# extreme_young = (
#     (df.age <= '24')
#     & ((df.code_exp == '20+')
#        | (df.ml_exp == '20+')
#        | (df.salary.isin(['300000-499999', '500000-999999']))
#       )
# )
# # extreme_young.sum()
# len(df[extreme_young])

In [ ]:
df = df[~extreme_young].reset_index(drop=True)

In [ ]:
len(df)

### Second easy candidate for invalid submissions:
Don't you think that something does not seem right with the salary distribution?
Logically, as we mentioned above beginners are expected to receive a lower salary for many reasons. But don't you think that for someone living in the USA, earing a yearly salary less than "2.8$", even for a part time job of, lets say 1 hour per day, is questionable for anything relating to coding?
Let us explote this a bit more; what could the chareteristics of the sample be that explain with such a low salary, especially for USA residents?
Obvious, candidates: level of education and experience.

In [ ]:
work_in_USA_for_3_per_day_code_exper = df[
    (df.salary.isin(['0-999'])
     & (df.country == 'United States of America'))
].groupby('code_exp', as_index=False).size()

work_in_USA_for_3_per_day_code_exper

Obviously, the 34 submissions with 20+ years of coding experience do not belong in the 0-999 salary range in the USA.


what should be the hourly wage for 40 hours a week for someone to get 1000 per year?
50 weeks * 40 hours = 2000 hours => hourly renumeration of 50 cents to get 1000.

If someone is part time employed = > 1000 hours per year => 1$ hourly wage.

If someone is part time eg 4 months = > 650 hours per year => 1.5$ hourly wage.

```
employent status   | year work  | usd per hour |
------------------------------------------------
full employed      | 2000 hours | 0.5          |     
part time (1/2)    | 1000 hours | 1            |
occasionally empl. | 650 hours  | 1.5          |
```
In general, someone with minimum 5, to 10 years of coding experience (a highly skilled individual in a demanding role!) that should be employed part time, e.g. only 5 hours per week, 20 hours per month, and only for 10 months, let's say for 5$ per hour, to be in this group. NOPE! No matter how you cut in it does not fit in.

In [ ]:
# A
work_in_USA_for_3_per_day_code_exper = df[
    (df.salary.isin(['0-999'])
     & (df.country == 'United States of America')
     & (df.code_exp.isin(['3-5', '5-10', '10-20', '20+']))
    )
].groupby(['code_exp', 'role'], as_index=False).size().sort_values('size', ascending=False)


# B
# work_in_USA_for_3_per_day_code_exper = df[(df.salary.isin(['0-999'])
#                                             & (df.country == 'United States of America')
#                                             & (df.code_exp.isin(['3-5', '5-10', '10-20', '20+']))
#                                           )
# ].groupby(['code_exp', 'role'], as_index=False).size().sort_values('size', ascending=False)

In [ ]:
#if you are not convinced, take a look at their submitted current occupation
# if you really like EDA, check the company employment size.
work_in_USA_for_3_per_day_code_exper = df[
    (df.salary.isin(['0-999'])
     & (df.country == 'United States of America'))
    & (df.code_exp.isin(['3-5', '5-10', '10-20', '20+']))
].groupby(['code_exp', 'role'], as_index=False).size().sort_values('size', ascending=False)

# work_in_USA_for_3_per_day_code_exper

#if you are not convinced, take a look at their submitted current occupation
# if you really like EDA, check the company employment size.
work_in_USA_for_3_per_day_code_exper = df[(df.salary.isin(['0-999'])
                                            & (df.country == 'United States of America'))
                                            & (df.code_exp.isin(['3-5', '5-10', '10-20', '20+']))
].groupby(['code_exp', 'role'], as_index=False).size().sort_values('size', ascending=False)

work_in_USA_for_3_per_day_code_exper

In [ ]:
# We definetely should drop these observations, if we intend to make meaningful EDA about this dataset.
# First, let's construct a similar condition for Ml experience.

In [ ]:
work_in_USA_for_3_per_day_ml_exper = df[(df.salary.isin(['0-999'])
                                          & (df.country == 'United States of America'))
].groupby('ml_exp', as_index=False).size()

work_in_USA_for_3_per_day_ml_exper

In [ ]:
work_in_India_for_3_per_day_code_exper = df[(df.salary.isin(['0-999'])
                                                       & (df.role != 'Currently not employed')
                                                       & (df.country == 'India'))
                                                     ].groupby('code_exp', as_index=False).size()

work_in_India_for_3_per_day_code_exper

In [ ]:
work_in_India_for_3_per_day_ml_exper = df[(df.salary.isin(['0-999'])
                                                     & (df.role != 'Currently not employed')
                                                     & (df.country == 'India'))
                                                   ].groupby('ml_exp', as_index=False).size()
# work_in_India_for_3_per_day_ml_exper

In [ ]:
# df.salary.unique()

In [ ]:
# len(df.salary.unique())

## ALSO SOS: We lose a lot of info with 25 bins, who are so unevenly distributed! A distribution chart may be misleading

### Since there are huge cross-country wage differences, we should set a reasonable and unamibguous threshold that would hold for all countries.
We could state that it is impossible for someone with 3 or more years of coding experience, or let's say 2 years of Ml experience to earn less than 80$ per month in such a skill demanding occupation, in any country (taking into account that this market offers a lot of opportunities for remote work on a global scale).

Since this threshold is ad hoc, we could be even more strict and set a the threshold to 2-3 years. This would not be a bad choice either.
Chosing a lower coding experience threshold would result in dropping more invalid submissions (True Positive, if we define Positive as identifying invalid submissions). But, we could ge also a few False Positives (dropping observations who ).
As a rule, in this part of the analysis, we will set a "loose" threshold, so as not to drop the maximum number of invalid submission, but in order to keep as many valid submissions as possible (minimize False Positives).

This analysis can be modified easily for stricter quality rules, as we will show in the end.
It would be equally reasonable to rule out any submissions in this salary range with more than 2 years of experience in either coding or machine learning, or even everybody who is currently employed in the USA, dropping perhpaps out of the data some fellows how less than part-time.

Also, note carefully that it would not be an optimal approach to use any statistical measures to exclude outliers for the data in this case, since all such measures are very biased by this kind of invalid submissions.

### To conclude, let us drop all submissions that stated that they are currently employed, with salary below $1000 and coding experience of 5 or more years or machine learning experience of more than 2 years.

In [ ]:
too_exper_for_min_salary = (
    (df.salary.isin(['0-999']))
    & (df.code_exp.isin(['3-5', '5-10', '10-20', '20+'])
       | (df.ml_exp.isin(['2-3', '3-4', '4-5', '5-10', '10-20', '20+']))
      )
)

len(df[too_exper_for_min_salary])

In [ ]:
# Example of invalid nonsense data
df.loc[16115][validation_cols]

In [ ]:
# question[107]

In [ ]:
df = df[~too_exper_for_min_salary].reset_index(drop=True)

In [ ]:
len(df), len(df[df.role == 'Data Scientist'])

In [ ]:
# Similarly, setting 'loose' thresholds that allow for
# some invalid submission to remain but
# minimize False Positives of invalid submissions:
too_exper_for_subsistence_salary = (
    (df.salary.isin(['1000-1999', '2000-2999', '3000-3999', '4000-4999']))
                             #  & (df.role != 'Currently not employed')  # not necessary condition
                               & ((df.code_exp.isin(['5-10', '10-20', '20+'])
                                  | (df.ml_exp.isin(['5-10', '10-20', '20+'])))
                            ))

len(df[too_exper_for_subsistence_salary])

In [ ]:
df = df[~too_exper_for_subsistence_salary].reset_index(drop=True)

In [ ]:
len(df), len(df[df.role == 'Data Scientist'])

# ???

In [ ]:
# Similarly, setting 'loose' thresholds that allow for
# some invalid submission to remain but
# minimize False Positives of invalid submissions:
too_exper_for_avg_salary = (
    #(df.salary.isin(['5000-7499', '7500-9999']))
        (df.salary.isin(['5000-7499']))
                               #& (df.role != 'Currently not employed')  # not necessary condition
                               & ((df.code_exp.isin(['10-20', '20+'])
                                  | (df.ml_exp.isin(['10-20', '20+'])))
                            ))

len(df[too_exper_for_avg_salary])

In [ ]:
df = df[~too_exper_for_avg_salary].reset_index(drop=True)

In [ ]:
# Similarly, setting 'loose' thresholds that allow for
# some invalid submission to remain but
# minimize False Positives of invalid submissions:
too_exper_for_low_salary = (
    (df.salary.isin(['10000-14999']))
                               #& (df.role != 'Currently not employed')  # not necessary condition
                               & ((df.code_exp.isin(['20+'])
                                  | (df.ml_exp.isin(['20+'])))  # perhaps add '10-20' in ml exp
                            ))

len(df[too_exper_for_low_salary])

In [ ]:
df = df[~too_exper_for_low_salary].reset_index(drop=True)

In [ ]:
len(df), len(df[df.role == 'Data Scientist'])

### Now let's work the other way around, no exper and young age and top salary

In [ ]:
# some invalid submission to remain but
# minimize False Positives of invalid submissions:
unexper_for_top_salary = (
    (df.salary.isin(['300000-499000', '500000-999999']))  # maybe add '300000-499000',
                               & (df.age <= '34')  
                               & (df.code_exp.isin(['1-2', '< 1', '0'])
                               & (df.ml_exp.isin(['1-2', '< 1', '0', np.nan]))  # perhaps or (instead of and) term
                            ))

len(df[unexper_for_top_salary])

In [ ]:
df_validation = df[validation_cols]

In [ ]:
demogr_unexper_for_top_salary = df_validation[unexper_for_top_salary]

In [ ]:
demogr_unexper_for_top_salary

In [ ]:
demogr_unexper_for_top_salary.index

In [ ]:
task_index = filter_df(df, 23)
# task_index

In [ ]:
task = task_index[task_index]
len(task)

In [ ]:
task.iloc[1]

In [ ]:
task.iloc[1][0]

In [ ]:
task.iloc[demogr_unexper_for_top_salary.index]  # task.iloc[['3088', '5612', '6573', '8319', '7820', '11348']]   # '7820'

# OBVIOUS INVALID CONDITIONS

#### Q5: Machine Learning Engineer VS Q15 "I do not use ML methods" : 'ml_exp' == 0
#### Q5: Machine Learning Engineer VS Q15 
#### Q5: Software Engineer VS Q6: 'code_exp' == 0
#### Q5: Data Engineer, Data Scientist, Data Analyst VS Q21: ds_team == 0 and Q23 Activities
#### Q15 "I do not use ML methods" : 'ml_exp' == 0 VS 
    A) Q16 anythin but 'None'
    B)Q17 using special methods such as   (this leads to Q18 and Q19)
    Gradient Boosting Machines (xgboost, lightgbm, etc)
    Dense Neural Networks (MLPs, etc)
    Convolutional Neural Networks
    Generative Adversarial Networks
    Recurrent Neural Networks
    Transformer Networks (BERT, gpt-3, etc)
    

In [ ]:
check_q23 = join_dfs(df_validation, task.iloc[df_validation.index])
check_q23.head(2)

In [ ]:
check_q23.columns[0:3]

In [ ]:
check_q23.iloc[0:3]

In [ ]:
 check_q23.iloc[:3, [1, 5]] 

In [ ]:
 check_q23.iloc[:3, [13]]

In [ ]:
check_q23[check_q23.columns[13]]

In [ ]:
check_q23.columns[13:16]

In [ ]:
check_q23.columns[[14]].values

In [ ]:
# ml_eng_without_exp = ((check_q23.role == 'Machine Learning Engineer')
#                         & ((check_q23.ml_exp == 0) 
#                            | (check_q23.ml_exp == np.nan)
#                           )
#                       & (
#                           #(check_q23['Build prototypes to explore applying machine learning to new areas'] == 1.0)
#                       | (check_q23[check_q23.columns[13]] == 1.0)
#                           | (check_q23[check_q23.columns[14]] == 1.0)
#                       | (check_q23[check_q23.columns[15]] == 1.0))
#                         #& (check_q23.iloc[:, [13]]  == '1.0')
#                      )

# check_q23[ml_eng_without_exp]

In [ ]:
data_role_no_team = df[
    ((df.role == 'Data Scientist')
     | (df.role == 'Data Engineer')
     | (df.role == 'Data Analyst'))
    & (df.team_ds == '0')
]

len(data_role_no_team)

In [ ]:
df[df.country == 'Poland']

In [ ]:
# SUGGESTIONS FOR QUESTIONNAIRE
# Q4: Should be 2 distinct questions
# Q5: about role could be 'Employed full time', 'Employed part time', 'Currently not employed', 'Student'
# and a Q6 for employment role with current question B choices.

In [ ]:
check_q23_activity_conditions = join_dfs(df_validation, task.iloc[df_validation.index])
check_q23_activity_conditions

In [ ]:
# demogr_task = join_dfs(demogr_unexper_for_top_salary, task)
# demogr_task

In [ ]:
# len(df.dropna(thresh=10))

In [ ]:
# len(df[df.duration < 61])

# IT IS DRAFT OR OLD CODE FROM HERE. IGNORE IT
Explore strange US salary above 100k to 200k and global modes of 10-15 and 40 to 50

In [ ]:
# salary_abmormal_India = ((df.salary.isin(['10000-14999'])  # 100000-124999', '125000-149999', '150000-199999', '10000-14999'
#                                & (df.country == 'India')
# #                                & (df.code_exp.isin(['10-20', '20+'])
#                                   #| (df.ml_exp.isin(['10-20', '20+'])))
#                        ))

# len(df[salary_abmormal_India])

In [ ]:
# df[salary_abmormal_India].groupby(['code_exp', 'age'], as_index=False).size()

In [ ]:
# too_exper_for_min_salary.sample(4)

In [ ]:
# extreme_young = ((df.salary.isin('') <= '24')
#                        & ((df.code_exp == '20+')
#                           | (df.ml_exp == '20+')
#                           | (df.salary.isin(['300000-499999', '500000-999999'])
#                          )
#                 ))

In [ ]:
# # create the user demogr_compgraphics dataframe
# demogr = df[['age', 'gender', 'country', 'education', 'role', 'code_exp', 'ml_exp', 'salary']]

In [ ]:
# demogr[
#     (demogr.salary.isin(['0-999'])
#     & (demogr.country == 'India'))
# ].groupby(['code_exp']).size().reset_index()

In [ ]:
# demogr[
#     (demogr.salary.isin(['0-999'])
#     & (demogr.country == 'India'))
# ].groupby(['code_exp']).size().reset_index()

In [ ]:
# demogr[
#     (demogr.salary.isin(['0-999'])
#     & (demogr.country == 'United States of America'))
# ].groupby(['code_exp', 'ml_exp']).size()#.reset_index()

In [ ]:
# us_in_group_code_exp = ds_Us_In.groupby(['country', 'code_exp', 'salary']).size().reset_index()
# us_in_group_code_exp.head(40)

In [ ]:
#Prepare Q23: Work activities, part of work role for analysis
# task = filter_df(df, 23)
# task = task[task]
# # task

In [ ]:
# demogr_task = join_dfs(demogr, task)
# demogr_task

In [ ]:
# # create the employment demogr_compgraphics dataframe
# comp = df[["Q20", "Q21", "Q22"]]
# comp.columns = ["empl_size", "ds_team", "comp_ml"]  # plus Q23 who we prepared
# # comp

In [ ]:
# demogr_comp = join_dfs(demogr_task, comp)

## EXAMINE Salary 'outliers"

Ideas: Clean the data from submission errors, not from Outliers.

A) Basic Variables:

    0) Absoluterly across all the data: Age and experience, experience and salary, age and salary (with country.) check if more than one 'extreme' answers.

    1) Within country: age + coding exper + ml exper + comp size + median or mean of salary of other bins (next and previous?, depending on No of Obs)
    Additive flow:
    Start with age and coding exper and ml exper and salary. Remove extremely obvious observations.
    2) Compare with other countries
    3) Compare with previous years
    4) salary size in comparison to bin and number of observations in the bin (eg. 500000 is 500 obs in first age bin and 1 in 10 in salaries bin)

B) Secondary Variables:

    1) Completion time:
    2) number of Nan:

In [ ]:
# demogr.salary.unique()

In [ ]:
# # too_much_exper_young = demo[((demo.age <= '22') & (demo.code_exp.isin(["10-20 years", "20+"])))]# too_much_exper_young = demo[((demo.age <= '22') & (demo.code_exp.isin(["5-10 years", "10-20 years", "20+"])))]
# extreme_young = demogr[((demogr.age <= "24") 
#                                  & ((demogr.code_exp > 20)  # > 10 = 10-20 years
#                                  |  (demogr.ml_exp > 20)  # > 10 = 10-20 years
#                                  |  (demogr.salary.isin(["300000-499999", "500000-999999"]))  # > 500
#                                    )
#                                 )]
# extreme_young

In [ ]:
# 27 observations should be excluded
# len(extreme_young)

In [ ]:
# less_exteme_younger = demogr[((demogr.age <= "21") 
#                                  & ((demogr.code_exp > 10) & (demogr.code_exp  < 30) # > 10 = 10-20 years
#                                  |  (demogr.ml_exp > 10) & (demogr.ml_exp  < 30) # > 10 = 10-20 years
#                                  |  (demogr.salary.isin(["300000-499999", "500000-999999"]))  # > 500
#                                    )
#                                 )]
# less_exteme_younger

In [ ]:
# len(exteme_young[exteme_young.salary.isin(["300000-499999", "500000-999999"])])

In [ ]:
# len(exteme_young[(exteme_young.salary == "500000-999999")
#                  | (exteme_young.salary == "300000-499999")
#                  & (exteme_young.country == "India")])

# len(exteme_young[(exteme_young.salary == "500000-999999")
#                  | (exteme_young.salary == "300000-499999")
#                  & (exteme_young.country == "India")
#                  & (exteme_young.role == "Data Scientist")])

# len(demogr[(demogr.salary == "500000-999999")
#            | (demogr.salary == "300000-499999")
#            & (demogr.country == "India")
#            & (demogr.role == "Data Scientist")])

# len(demogr[(demogr.Q24 == "500000-999999") & (demogr.Q3 == "India")])

# len(orig[(orig.Q24 == "> $500,000")])# & (orig.Q3 == "India")])

# len(orig[(orig.Q24 == "> $500,000") & (orig.Q3 == "India")])

# orig.Q4

# data_scientists_India = demogr_comp[(
#      # (demogr_comp.role =='Data Scientist')
#      (demogr_comp.country.isin(['India']))
#     & (demogr_comp.salary.notna())
#     & ((demogr_comp.salary == "500000-999999")
#     |  (demogr_comp.salary == "300000-499999"))
#     #& (demogr_comp.code_exp <= 10)
# )].reset_index(drop=True)
# data_scientists_India.sort_values(by='salary').head(3)

# # demogr_comp['combined_exp'] = demogr_comp['code_exp'] + demogr_comp['ml_exp']

# # demogr_comp.role.value_counts()

# # demogr_comp[demogr_comp.role == 'Currently not employed']#.value_counts()

In [ ]:
# too_much_exper_young = demogr_comp[((demogr_comp.age <= '24') & 
#                              (demogr_comp.role == 'data_scientist') & 
#                              (demogr_comp.code_exp.isin(["20+ years"]) |
#                               demogr_comp.ml_exp.isin(["20 or more years"]) |
#                               demogr_comp.salary > 100000)
#                             )]
# too_much_exper_young

In [ ]:
# demogr_comp.country.unique()

In [ ]:
# ds = demogr_comp[demogr_comp['role'] == 'Data Scientist']
# ds

In [ ]:
# ds.country.value_counts()

In [ ]:
# demogr_comp.ml_exp.value_counts()

# data_scientists_USA = demogr_comp[(
#       (demogr_comp.role =='Data Scientist')
#     & (demogr_comp.country.isin(['United States of America']))
#     & (demogr_comp.salary.notna())
#     & (demogr_comp.salary > 199000 )
#     & (demogr_comp.salary <= 500000)
#     & (demogr_comp.code_exp <= 10)
# )].reset_index(drop=True)
# data_scientists_USA.sort_values(by='salary').head(3)

# code_exp_salary = data_scientists_USA.groupby('code_exp').size()
# code_exp_salary

# code_exp_salary = data_scientists_USA.groupby('code_exp').mean()  # .size()#
# code_exp_salary

# plt.plot(code_exp_salary.salary)

# ml_exp_salary = data_scientists_USA.groupby('ml_exp').mean()  #.size()
# ml_exp_salary

# plt.plot(ml_exp_salary.salary)

# data_scientists_USA.salary.value_counts()

# data_scientists_USA.salary.dtypes

In [ ]:
# data_scientists_USA['salary'] = pd.to_numeric(data_scientists_USA['salary'], errors='coerce')

In [ ]:
# data_scientists_USA['salary'] = data_scientists_USA['salary'].astype(float)

In [ ]:
# five_year_exper_data_scientists_USA = demogr_comp[(
#       (demogr_comp.role =='Data Scientist')
#     & (demogr_comp.country.isin(['United States of America']))
#     & (demogr_comp.code_exp == 30)
#     & (demogr_comp.ml_exp == 20)
#     #| (demogr_comp.salary > 125000)
# )]
# five_year_exper_data_scientists_USA.salary.mean()

# five_year_exper_data_scientists_USA = demogr_comp[(
#       (demogr_comp.role =='Data Scientist')
#     & (demogr_comp.country.isin(['United States of America']))
#     & (demogr_comp.code_exp == 30)
#     & (demogr_comp.ml_exp == 10)
#     #| (demogr_comp.salary > 125000)
# )]
# five_year_exper_data_scientists_USA.salary.mean()

# five_year_exper_data_scientists_USA = demogr_comp[(
#       (demogr_comp.role =='Data Scientist')
#     & (demogr_comp.country.isin(['United States of America']))
#     & (demogr_comp.code_exp == 20)
#     & (demogr_comp.ml_exp == 5)
#     #| (demogr_comp.salary > 125000)
# )]
# five_year_exper_data_scientists_USA.salary.mean()

# five_year_exper_data_scientists_USA = demogr_comp[(
#       (demogr_comp.role =='Data Scientist')
#     & (demogr_comp.country.isin(['United States of America']))
#     & (demogr_comp.code_exp == 10)
#     & (demogr_comp.ml_exp == 5)
#     #| (demogr_comp.salary > 125000)
# )]
# five_year_exper_data_scientists_USA.salary.mean()

# five_year_exper_data_scientists_USA = demogr_comp[(
#       (demogr_comp.role =='Data Scientist')
#     & (demogr_comp.country.isin(['United States of America']))
#     & (demogr_comp.code_exp == 5)
#     & (demogr_comp.ml_exp == 5)
#     #| (demogr_comp.salary > 125000)
# )]
# five_year_exper_data_scientists_USA.salary.mean()

# five_year_exper_data_scientists_USA = demogr_comp[(
#       (demogr_comp.role =='Data Scientist')
#     & (demogr_comp.country.isin(['United States of America']))
#     & (demogr_comp.code_exp == 5)
#     & (demogr_comp.ml_exp == 4)
#     #| (demogr_comp.salary > 125000)
# )]
# five_year_exper_data_scientists_USA.salary.mean()

# five_year_exper_data_scientists_USA = demogr_comp[(
#       (demogr_comp.role =='Data Scientist')
#     & (demogr_comp.country.isin(['United States of America']))
#     & (demogr_comp.code_exp == 5)
#     & (demogr_comp.ml_exp == 3)
#     #| (demogr_comp.salary > 125000)
# )]
# five_year_exper_data_scientists_USA.salary.mean()

In [ ]:
# ten_year_exper_data_scientists_USA = demogr_comp[(
#       (demogr_comp.role =='Data Scientist')
#     & (demogr_comp.country.isin(['United States of America']))
#     & (demogr_comp.code_exp == 5)
#     & (demogr_comp.ml_exp == 2)
#     #| (demogr_comp.salary > 125000)
# )]
# ten_year_exper_data_scientists_USA.salary.mean()

# twenty_year_exper_data_scientists_USA = demogr_comp[(
#       (demogr_comp.role =='Data Scientist')
#     & (demogr_comp.country.isin(['United States of America']))
#     & (demogr_comp.code_exp == 20)
#     & (demogr_comp.ml_exp == 20)
#     #| (demogr_comp.salary > 125000)
# )]
# twenty_year_exper_data_scientists_USA.salary.mean()

# demogr_25_29 = demogr_comp[(demogr_comp['age'] == '25-29')]
# ds_Us_In = ds[
#       (ds['country'] == 'United States of America')
#     | (ds['country'] == 'India')].reset_index(drop=True)
# ds_Us_In

# # len(five_year_exper_data_scientists_USA_India)

# # five_year_exper_data_scientists_USA_India.groupby("country").mean()

# ds.info()

# ds.ml_exp.unique()

# ds.code_exp.value_counts().sort_index()

# ds.ml_exp.value_counts().sort_index()

# demogr[demogr.Q5 == "Data Scientist"].Q24.describe()

# ds.country.value_counts().sort_index()

# ds.age.value_counts(True).sort_index()

# ds.gender.value_counts(True).sort_index()

# # ds.country.value_counts(sort=True)#.sort_index()



# len(ds_Us_In.salary)

# ds_Us_In.salary.value_counts().sort_index()

# ds_Us_In.code_exp.value_counts().sort_index()

# ds_Us_In.ml_exp.value_counts().sort_index()

# ds_Us_In

# us_in_group_code_exp = ds_Us_In.groupby(['country', 'code_exp']).mean()
# us_in_group_code_exp

# us_in_group_code_exp = ds_Us_In.groupby(['country', 'education','code_exp', 'ml_exp', 'salary']).size().reset_index()
# us_in_group_code_exp.head(40)

# us_in_group_code_exp = ds_Us_In.groupby(['country', 'code_exp', 'salary']).size().reset_index()
# us_in_group_code_exp.head(40)

# us_in_group_code_exp[us_in_group_code_exp['country'] == 'India']

# ds_Us_In[ds_Us_In['salary'] > 999000]

# ds_Us_In[ds_Us_In['salary'] == 500000]

# too_much_exper_young = demogr_comp[((demogr_comp.age <= '24') & 
#                              (demogr_comp.code_exp.isin(["20+ years"]) |
#                               demogr_comp.ml_exp.isin(["20 or more years"]) |
#                               demogr_comp.salary.isin(["199.999", "249.999", "299.999", "500.000"]))
#                             )]
# too_much_exper_young

